In [1]:
def create_dataset_if_not_exists(client, dataset_id, location="US"):
    """Cria o dataset no BigQuery se ele não existir."""
    from google.cloud import bigquery

    dataset_ref = bigquery.DatasetReference(client.project, dataset_id)
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = location
    try:
        client.get_dataset(dataset_ref)  # Verifica se o dataset já existe
        print(f"Dataset {dataset_id} já existe.")
    except Exception:
        print(f"Dataset {dataset_id} não encontrado. Criando dataset...")
        client.create_dataset(dataset)
        print(f"Dataset {dataset_id} criado com sucesso.")

In [2]:
def create_table_if_not_exists(client, dataset_id, table_id, schema=None):
    """Cria a tabela no BigQuery se ela não existir."""
    from google.cloud import bigquery
    
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    try:
        client.get_table(table_ref)  # Tenta obter a tabela
        print(f"Tabela {dataset_id}.{table_id} já existe.")
    except Exception:
        print(f"Tabela {dataset_id}.{table_id} não existe. Criando tabela...")
        # Cria a tabela com o esquema detectado
        table = bigquery.Table(table_ref, schema=schema)
        table = client.create_table(table)
        print(f"Tabela {dataset_id}.{table_id} criada com sucesso.")

In [3]:
def define_schema_from_dataframe(df):
    """Define o esquema do BigQuery com base no DataFrame pandas."""
    import pandas as pd
    from google.cloud import bigquery
    
    schema = []
    for column, dtype in zip(df.columns, df.dtypes):
        if pd.api.types.is_integer_dtype(dtype):
            field_type = "INTEGER"
        elif pd.api.types.is_float_dtype(dtype):
            field_type = "FLOAT"
        elif pd.api.types.is_bool_dtype(dtype):
            field_type = "BOOLEAN"
        else:
            field_type = "STRING"  # Inclui tipos como objetos e strings

        schema.append(bigquery.SchemaField(column, field_type))
    return schema

In [9]:
def upload_multiple_csv_to_bigquery(dataset_id, table_id, csv_files):
    import pandas as pd
    from google.cloud import bigquery

    client = bigquery.Client()

    # Carrega o primeiro CSV como DataFrame para definir o esquema e criar a tabela
    first_file_path = csv_files[0]
    df = pd.read_csv(first_file_path)
    df = df.astype({' Started': str, ' Ends': str})
    schema = define_schema_from_dataframe(df)

    # Cria o dataset, se ainda não existir
    create_dataset_if_not_exists(client, dataset_id)

    # Cria a tabela, se ainda não existir
    create_table_if_not_exists(client, dataset_id, table_id, schema=schema)

    # Carrega cada arquivo CSV para a mesma tabela
    for csv_file_path in csv_files:
        df = pd.read_csv(csv_file_path)  # Carrega o CSV em um DataFrame
        
        # Envia o DataFrame para o BigQuery
        client.load_table_from_dataframe(
            df, 
            f"{dataset_id}.{table_id}",
            job_config=bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
        ).result()
        print(f"Arquivo {csv_file_path} enviado com sucesso.")

In [10]:
import os, glob

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "configs/credentials.json"

csv_files = glob.glob("raw/*.csv")
dataset_id = "beAnalytics"
table_id = "steam_sales"
upload_multiple_csv_to_bigquery(dataset_id, table_id, csv_files)

Dataset beAnalytics não encontrado. Criando dataset...
Dataset beAnalytics criado com sucesso.
Tabela beAnalytics.steam_sales não existe. Criando tabela...
Tabela beAnalytics.steam_sales criada com sucesso.
Arquivo raw/steamdb_sales_page_19.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_4.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_11.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_2.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_15.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_6.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_5.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_10.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_14.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_12.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_18.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_1.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_3.csv enviado com sucesso.
Arquivo raw/steamdb_sales_page_7